In [1]:
import psycopg2
from psycopg2.extras import DictCursor

In [2]:
def connect():
    return psycopg2.connect(dbname='postgres', user='postgres', password='coursework')

In [3]:
conn = connect()

In [4]:
def purge_items(conn):
    with conn.cursor(cursor_factory=DictCursor) as cursor:
        truncate = "TRUNCATE shopping_list"
        cursor.execute(truncate)
        conn.commit()
    return

In [5]:
purge_items(conn)

In [6]:
def get_items(conn, user_id):
    with conn.cursor(cursor_factory=DictCursor) as cursor:
        select = f"SELECT product, quantity FROM shopping_list WHERE user_id = '{user_id}'" + \
                 f"AND quantity <> 0;"
        cursor.execute(select)
        records = cursor.fetchall()
    return records

In [7]:
print(get_items(conn, 1))
records = list(zip(*get_items(conn, 1)))
try:
    shopping_list = list(records[0])
    quantity_list = list(records[1])
except:
    shopping_list = []
    quantity_list = []
shopping_list, quantity_list

[]


([], [])

In [8]:
def add_items(conn, user_id, products, quantity):
    with conn.cursor(cursor_factory=DictCursor) as cursor:
        rows = list(zip(*(products, quantity)))
        VALUES = ', '.join(f"('{user_id}', '{row[0]}', {row[1]}, current_timestamp(0))" for row in rows)
        insert = f"""INSERT INTO 
                        shopping_list (user_id, product, quantity, created_on)
                     VALUES 
                        {VALUES}
                     ON CONFLICT ON CONSTRAINT shopping_list_user_id_product_key DO UPDATE
                        SET quantity = shopping_list.quantity + excluded.quantity,
                            created_on = current_timestamp(0);"""
        cursor.execute(insert)
        conn.commit()
    return

In [9]:
add_items(conn, 1, ['milk', 'bread', 'eggs', 'tea', 'noodles'], [2, 1, 3, 4, 1])

In [10]:
def del_items(conn, user_id, products, quantity):
    with conn.cursor(cursor_factory=DictCursor) as cursor:
        rows = list(zip(*(products, quantity)))
        CONDITIONS = '\n'.join(f"WHEN product = '{row[0]}'" +
                               f"THEN GREATEST(quantity - {row[1]}, 0)" for row in rows)
        update = f"""UPDATE shopping_list
                     SET quantity = CASE
                        {CONDITIONS}
                        ELSE quantity
                     END
                     WHERE user_id = '{user_id}';"""
        cursor.execute(update)
        conn.commit()
    return

In [11]:
del_items(conn, 1, ['milk', 'eggs', 'tea'], [2, 2, 3])

In [12]:
get_items(conn, 1)

[['bread', 1], ['eggs', 1], ['tea', 1], ['noodles', 1]]